In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("/Users/suresh/Downloads/sentences.csv",sep='\t')

In [3]:
data.columns = ['Sno','Language','Text']
data.head()

,Sno,Language,Text
0,2,cmn,我该去睡觉了。
1,3,cmn,你在干什麼啊？
2,4,cmn,這是什麼啊？
3,5,cmn,今天是６月１８号，也是Muiriel的生日！
4,6,cmn,生日快乐，Muiriel！


In [4]:
data = data[data['Language']=='eng']
data = data[['Sno','Language','Text']]

In [5]:
data['Sno'] = list(data.index)

In [6]:
data.shape[0]

1234502

In [7]:
data.head()

,Sno,Language,Text
1231,1231,eng,Let's try something.
1232,1232,eng,I have to go to sleep.
1234,1234,eng,Today is June 18th and it is Muiriel's birthday!
1235,1235,eng,Muiriel is 20 now.
1236,1236,eng,"The password is ""Muiriel""."


In [8]:
import math
range(0,math.ceil(data.shape[0]/10))

range(0, 123451)

In [9]:
def generate_text(all_sentences=None):
    data_tagged = []
    comb_sents_num = 2
    num_texts = math.ceil(len(all_sentences)/comb_sents_num)
    
    for i in range(0,num_texts):
      #print(i)      
      l_limit = i * comb_sents_num
      if (i*comb_sents_num) + comb_sents_num > num_texts:
        u_limit = num_texts
      else:    
        u_limit = (i*comb_sents_num) + comb_sents_num
      sentence_i = 0
      d = []
      for each_sent in all_sentences[l_limit:u_limit]:
              
        words = each_sent.split(" ")
        words[-1] = re.sub('[^0-9a-zA-Z]+', '', words[-1])
        tags = ['neos']*(len(words)-1)
        tags.append('eos')
        text_num = ['Text '+str(i)]*len(words)
        sentence_num = ['Sentence '+str(sentence_i)]*len(words)
        d.extend([(w,t) for w,t in zip(words,tags)])
        data_tagged.append(d)
        sentence_i = sentence_i+1
    return data_tagged        

In [10]:
texts = generate_text(data['Text'].values.tolist())

In [11]:
texts[0][0:10]

[("Let's", 'neos'),
 ('try', 'neos'),
 ('something', 'eos'),
 ('I', 'neos'),
 ('have', 'neos'),
 ('to', 'neos'),
 ('go', 'neos'),
 ('to', 'neos'),
 ('sleep', 'eos')]

In [12]:
max_len = 50
tag2idx = {t: i for i, t in enumerate(['neos','eos'])}

In [13]:
tag2idx

{'eos': 1, 'neos': 0}

In [14]:
X = [[w[0] for w in t] for t in texts]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

In [15]:
print(X[0][:10])
print(X[0][-10:-1])

["Let's", 'try', 'something', 'I', 'have', 'to', 'go', 'to', 'sleep', '__PAD__']
['__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__', '__PAD__']


In [16]:
y = [[tag2idx[w[1]] for w in t] for t in texts]

In [17]:
del data
del new_X
#del w
#del t
#del new_seq
del texts

In [18]:
from keras.preprocessing.sequence import pad_sequences
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["neos"])

Using TensorFlow backend.


In [19]:
y[0][-10:-1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=2019)

In [22]:
del X
del y

In [23]:
batch_size = 32

In [24]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

In [25]:
tf.__version__

'1.15.0-rc3'

In [26]:
sess = tf.Session()
K.set_session(sess)

In [27]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [28]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [29]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [30]:
input_text = Input(shape=(max_len,), dtype='string')
embedding = Lambda(ElmoEmbedding, output_shape=(None, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(2, activation="softmax"))(x)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [31]:
model = Model(input_text, out)

In [32]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [33]:
len(X_tr)

555525

In [34]:
X_tr, X_val = X_tr[:(int(len(X_tr)/batch_size))*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:int(len(X_tr)/batch_size)*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

In [35]:
print (len(X_val))
print (len(X_tr))

4320
555520


In [36]:
import numpy as np

In [ ]:
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size, epochs=2, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 555520 samples, validate on 4320 samples
Epoch 1/2
339456/555520 [=================>............] - ETA: 42:47:45 - loss: 0.0014 - accuracy: 0.9996

In [ ]:
32*5